## Linear Regression
An estimator is any model that tries to estimate a variable y from another variable(s) x given pairs of data (x<sub>1</sub>,y<sub>1</sub>),(x<sub>2</sub>,y<sub>2</sub>),...,(x<sub>N</sub>,y<sub>N</sub>)

Regression is when the targets (y) are quantities (not cat vs dog but rather price of a house)

An example would be trying to guess how many people will buy icecream from a shop at any day given the temprature on that day. In this case, the target is the number of icecream people will buy, it is an integer and any value (ex: 14 icecreams) doesn't represent a particular class, so it will be represented as a regression problem. In this case, it turns out that x is also a quantity (temprature); however, this is not necessary and is tackled by approaches other than linear regression.

To estimate the price of the icecream from temperature we gather the data on various days from various different places and get the following data.



We can see that a line (could  also be a curve) could fit the data, but how can we find which line exactly.

#### Line Equation Review
A line in 2D can be parametrized using a slope m and a y-intercept b which would result in it having an equation

$$y = mx+b$$

In the example of the icecream, if the temperature at a day was 30, our prediction for the number of icecreams sold would be m\*30 + b, so we simply need to find m and b.

#### Squared Loss
We need some measurement of how good our line is to be able to find the "best" line, so we will measure the difference between our predictions and the correct values from the data.


Then, we will square all the differences and add them up. This will be the value we try to minimize.

#### Note
The normal equation will probably not work. It is left for you to figure it out and let everyone else know, let's see who figures it out first (it's a tricky issue).




In [1]:
import numpy as np
import pandas as pd

In [2]:
class Linear_Regression():
    '''
    Linear Regression model created using only NumPy
    
    Attributes
    ----------
    weights: np.array of floats
        All the parameters of the model (including bias)
    '''
    def __init__(self):
        self.weights = None
        
    def train(self,data_X,data_y):
        '''
        Train the model using the given data
        
        Parameters
        ----------
        data_X: np.array, shape = (N, num_features)
            Features from the data, each row is one data point. Assumes that a column of ones was added to data_X
        data_y: np.array, shape = (N, num_targets)
            The target values to predict, each row contains the targets for one data point
        '''
        ########################## Insert code here ##########################
        self.weights = (np.linalg.inv(data_X.T@data_X).dot(data_X.T@data_y).T)
        
    def predict(self,x_to_predict):
        '''
        Predict using the given value as input
        
        Assumes that self.train(.,.) has been called before calling this method
        
        Parameters
        ----------
        x_to_predict: np.array, shape = (M, num_features)
            A given list of inputs to predict targets for, each row is one input. Assumes that a column of ones was added similar to the training data
        
        Returns
        -------
        np.array of floats, shape = (M, num_targets)
            Predicted values for each input
        '''
        w = self.weights.reshape((74,1)).T
        ########################## Insert code here ##########################
        return w.dot(x_to_predict.T)

### Import the data and remove useless columns

In [3]:
df = pd.read_csv("train.csv")
df.drop(columns=["Id"],inplace=True)
df.shape

(1460, 80)

### Handle the missing data (NaNs)

In [4]:
df.drop(columns=df.columns[df.isnull().sum().values>200],inplace=True)
df.dropna(inplace=True)
df.shape

(1338, 74)

### Replace categorical data (strings) with numerical values

In [5]:
obj_to_replace = df["MSZoning"].dtype

for column in df.columns:
    if df[column].dtype == obj_to_replace:
        uniques = np.unique(df[column].values)
        for idx,item in enumerate(uniques):
            df[column] = df[column].replace(item,idx)
            
df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,3,8450,1,3,3,0,4,0,5,...,0,0,0,0,0,2,2008,8,4,208500
1,20,3,9600,1,3,3,0,2,0,24,...,0,0,0,0,0,5,2007,8,4,181500
2,60,3,11250,1,0,3,0,4,0,5,...,0,0,0,0,0,9,2008,8,4,223500
3,70,3,9550,1,0,3,0,0,0,6,...,272,0,0,0,0,2,2006,8,0,140000
4,60,3,14260,1,0,3,0,2,0,15,...,0,0,0,0,0,12,2008,8,4,250000


### Add the bias column (column of ones)

In [6]:
df["bias"] = np.ones(df.shape[0])
df.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,bias
0,60,3,8450,1,3,3,0,4,0,5,...,0,0,0,0,2,2008,8,4,208500,1.0
1,20,3,9600,1,3,3,0,2,0,24,...,0,0,0,0,5,2007,8,4,181500,1.0
2,60,3,11250,1,0,3,0,4,0,5,...,0,0,0,0,9,2008,8,4,223500,1.0
3,70,3,9550,1,0,3,0,0,0,6,...,0,0,0,0,2,2006,8,0,140000,1.0
4,60,3,14260,1,0,3,0,2,0,15,...,0,0,0,0,12,2008,8,4,250000,1.0


### Divide the data into training, testing, X, and y

In [7]:
df = df.sample(frac=1).reset_index(drop=True)
training_df = df[:-100]
val_df = df[-100:]
training_y = training_df["SalePrice"].values
training_X = training_df.drop(columns=["SalePrice"]).values
val_y = val_df["SalePrice"].values
val_X = val_df.drop(columns=["SalePrice"]).values

print(training_X.shape)
print(training_y.shape)

(1238, 74)
(1238,)


### Train the linear regressor

In [8]:
# Create and fit the model
LR_regressor = Linear_Regression()
LR_regressor.train(training_X,training_y)
# Calculate Mean Absolute Error (Easier to interpret than MSE)
########################## Insert code here ##################

#import the testing data
testing_df = pd.read_csv("test.csv")
#remove the ID column
testing_df.drop(columns=["Id"],inplace=True)
#remove NaNs fields
testing_df.drop(columns=testing_df.columns[testing_df.isnull().sum().values>200],inplace=True)
testing_df.dropna(inplace=True)

#replaces string values with int
for column in testing_df.columns:
    if testing_df[column].dtype == obj_to_replace:
        uniques = np.unique(testing_df[column].values)
        for idx,item in enumerate(uniques):
            testing_df[column] = testing_df[column].replace(item,idx)
            
#Add the bias column and set its value to 1          
testing_df["bias"] = np.ones(testing_df.shape[0])
# y1_predictions == > predictions for the testing set
y1_predictions = np.absolute(np.array(LR_regressor.predict(testing_df)))
# y2_predictions == > predictions for the training set, to see how close the model is to the actual prices
y2_predictions = np.absolute(np.array(LR_regressor.predict(training_X)))

print(y1_predictions.mean())
print(y2_predictions.mean())
MAE = np.absolute(y2_predictions - training_y)/ 1238.0
print(MAE) #HEEEELP! That doesn't make any sense

15555266.60638593
15869699.507893147
[[ 6509.87418378  7173.50407287 26854.94132888 ... 14296.67695504
   7718.03472149  5325.51760267]]


### Train using the sklearn linear regressor

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Create and fit the model
########################## Insert code here ##########################
reg = LinearRegression().fit(training_X, training_y)
print(reg.score(training_X, training_y))
sci_prediction = reg.predict(training_X)
# Calculate Mean Absolute Error (Easier to interpret than MSE)
########################## Insert code here ##########################
MAE = mean_absolute_error(y_true=(sci_prediction),y_pred=training_y)
print(MAE) #This Makes Sense

0.8758989406049058
17428.941108598025
